<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hu_whisper_medium_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-b2a2p7zf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-b2a2p7zf
  Resolved https://github.com/huggingface/transformers to commit 2b5d5ead53207963fc57fb26cb3e6c77dd95d01a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8125115 sha256=ebfec549d966469b4b44bbc2e03100bff6456698618f874ec3b4f8c1fb307b7f
  Stored in directory: /tmp/pip-ephem-wheel-cache-17np9qxx/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hu", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1110
})

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=25507a80e41874923fd410d73726ac7036bb08ce3a8a87882d4960943435b9c3
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [19]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:20<00:00, 75.9MiB/s]


In [20]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [21]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [22]:
text

,0
0,ezenközben romániában egy környezeti és termés...
1,akkor amikor azt tapasztaljuk hogy ezrek halna...
2,azt gondolom hogy nagyon fontos azt látni hogy...
3,európai szinten van szükségünk adatokra mert c...
4,és gratulálok az észtek első elnökségének is k...
...,...
1105,azt gondolom hogy ideje megtennünk néhány megá...
1106,el kell érnünk hogy a jövőben az új lakóépület...
1107,én mindenképpen úgy gondolom az elmúlt időszak...
1108,elnök asszony tisztelt biztos úr képviselőtárs...


In [23]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [24]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,ezenközben romániában egy környezeti és termés...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0006103515625, 0.001617431640625, 0.002746...",16000
1,male,akkor amikor azt tapasztaljuk hogy ezrek halna...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.025787353515625, 0.03045654296875, 0.006683...",16000
2,male,azt gondolom hogy nagyon fontos azt látni hogy...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, -0.001068115234375, -0.001...",16000
3,female,európai szinten van szükségünk adatokra mert c...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00030517578125, 0.001617431640625, -0.00103...",16000
4,female,és gratulálok az észtek első elnökségének is k...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.042938232421875, -0.039215087890625, -0.04...",16000
...,...,...,...,...,...
1105,male,azt gondolom hogy ideje megtennünk néhány megá...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0001220703125, -0.001434326171875, -6.10351...",16000
1106,female,el kell érnünk hogy a jövőben az új lakóépület...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.011688232421875, 0.001251220703125, 0.0048...",16000
1107,female,én mindenképpen úgy gondolom az elmúlt időszak...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.01953125, -0.0189208984375, -0.02023315429...",16000
1108,male,elnök asszony tisztelt biztos úr képviselőtárs...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000213623046875, -0.00018310546875, 0.0, -...",16000


In [25]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,ezenközben romániában egy környezeti és termés...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0006103515625, 0.001617431640625, 0.002746...",16000
1,male,akkor amikor azt tapasztaljuk hogy ezrek halna...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.025787353515625, 0.03045654296875, 0.006683...",16000
2,male,azt gondolom hogy nagyon fontos azt látni hogy...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, -0.001068115234375, -0.001...",16000
10,male,én mint magyar nem kérek ebből a szégyenből.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.136932373046875, -0.26202392578125, -0.131...",16000
11,male,az európai konzervatvok az emberi méltóság hve...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00018310546875, -0.000457763671875, -0.000...",16000
...,...,...,...,...,...
1100,male,új hirdetést javaslok a szlovák vendéglátóipar...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00042724609375, 0.000640869140625, 0.00112...",16000
1101,male,nem csak nemzetgazdaságaink hanem az uniós pol...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00048828125, 0.0, 0.00030517578125, 0.0009...",16000
1102,male,az ön kormánya jogot szerzett arra hogy belené...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0001220703125, -0.00091552734375, -0.002532...",16000
1105,male,azt gondolom hogy ideje megtennünk néhány megá...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0001220703125, -0.001434326171875, -6.10351...",16000


In [26]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
3,female,európai szinten van szükségünk adatokra mert c...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00030517578125, 0.001617431640625, -0.00103...",16000
4,female,és gratulálok az észtek első elnökségének is k...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.042938232421875, -0.039215087890625, -0.04...",16000
5,female,az hogy az európai bizottság megválasztott eln...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.116912841796875, -0.138214111328125, -0.13...",16000
6,female,úr! biztos asszony! kedves képviselőtársaim! s...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001800537109375, -0.0023193359375, -0.0043...",16000
7,female,a tavares jelentésnek egyébként többek között ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001251220703125, 0.001007080078125, 0.00027...",16000
...,...,...,...,...,...
1103,female,az előttünk lévő irányelv a tömegek számára kv...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00494384765625, -0.000518798828125, -0.0033...",16000
1104,female,az energiaszektorban a beruházások és a beruhá...,/root/.cache/huggingface/datasets/downloads/ex...,"[-9.1552734375e-05, 0.000396728515625, -0.0001...",16000
1106,female,el kell érnünk hogy a jövőben az új lakóépület...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.011688232421875, 0.001251220703125, 0.0048...",16000
1107,female,én mindenképpen úgy gondolom az elmúlt időszak...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.01953125, -0.0189208984375, -0.02023315429...",16000


In [27]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hu")
    return result["text"]

In [28]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [29]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.3333333333333333
Processed 2 files, WER: 0.3181818181818182
Processed 3 files, WER: 0.47368421052631576
Processed 4 files, WER: 0.375
Processed 5 files, WER: 0.65
Processed 6 files, WER: 0.16666666666666666
Processed 7 files, WER: 0.38461538461538464
Processed 8 files, WER: 0.4444444444444444
Processed 9 files, WER: 0.1875
Processed 10 files, WER: 0.21052631578947367
Processed 11 files, WER: 0.2647058823529412
Processed 12 files, WER: 0.3333333333333333
Processed 13 files, WER: 0.3333333333333333
Processed 14 files, WER: 0.23529411764705882
Processed 15 files, WER: 0.35
Processed 16 files, WER: 0.35294117647058826
Processed 17 files, WER: 0.4117647058823529
Processed 18 files, WER: 0.32432432432432434
Processed 19 files, WER: 0.3333333333333333
Processed 20 files, WER: 0.0
Processed 21 files, WER: 0.3125
Processed 22 files, WER: 0.32142857142857145
Processed 23 files, WER: 0.16666666666666666
Processed 24 files, WER: 0.3333333333333333
Processed 25 files, WER:

In [18]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.782608695652174
Processed 2 files, WER: 0.7647058823529411
Processed 3 files, WER: 1.0
Processed 4 files, WER: 0.75
Processed 5 files, WER: 0.8235294117647058
Processed 6 files, WER: 2.018867924528302
Processed 7 files, WER: 0.6875
Processed 8 files, WER: 0.6944444444444444
Processed 9 files, WER: 1.037037037037037
Processed 10 files, WER: 0.8823529411764706
Processed 11 files, WER: 1.0
Processed 12 files, WER: 0.8888888888888888
Processed 13 files, WER: 0.8
Processed 14 files, WER: 1.0
Processed 15 files, WER: 1.1666666666666667
Processed 16 files, WER: 0.7862068965517242
Processed 17 files, WER: 1.0
Processed 18 files, WER: 1.0
Processed 19 files, WER: 1.0
Processed 20 files, WER: 0.9285714285714286
Processed 21 files, WER: 0.9444444444444444
Processed 22 files, WER: 1.0
Processed 23 files, WER: 1.1666666666666667
Processed 24 files, WER: 0.9130434782608695
Processed 25 files, WER: 0.9090909090909091
Processed 26 files, WER: 0.5
Processed 27 files, WER: 0.96